# Ionization states

The ionization state distribution for an element refers to the fractions of that element at each ionic level.  For example, the charge state of helium might be 10% He$^{0+}$, 70% He$^{1+}$, and 20% He$^{2+}$. 

In [1]:
from plasmapy.particles import IonizationState, IonizationStateCollection

## The ionization state of a single element

Let's store this information in an `IonizationState` object, and specify the number density of the element too.

In [3]:
He_states = IonizationState("He-4", [0.1, 0.7, 0.2], n_elem=1e7 * u.cm ** -3)

<IonizationState instance for He-4>


The ionization state distribution is stored in the `ionic_fractions` attribute.

In [4]:
He_states.ionic_fractions

array([0.1, 0.7, 0.2])

We can get the symbols for each ionic level too.

In [7]:
He_states.ionic_symbols

['He-4 0+', 'He-4 1+', 'He-4 2+']

Because we provided the number density of the element as a whole, we can get back the number density of each ionic level.

In [8]:
He_states.number_densities

<Quantity [1.e+12, 7.e+12, 2.e+12] 1 / m3>

We can also get the electron number density required to balance the positive charges for ions of this element.

In [9]:
He_states.n_e

<Quantity 1.1e+13 1 / m3>

We can provide an `IonizationState` with a charge number as an index to get an `IonicLevel` object that contains most of these attributes, but for a single ionic level (like He$^{1+}$).  This capability is useful if we wish to iterate over the ions of an element.

In [13]:
for Z in range(3):
    print(He_states[Z])

IonicLevel('He-4 0+', ionic_fraction=0.1)
IonicLevel('He-4 1+', ionic_fraction=0.7)
IonicLevel('He-4 2+', ionic_fraction=0.2)


We can get information about the average charge state via `Z_mean`, `Z_most_abundant`, and `Z_rms`.

In [14]:
He_states.Z_mean

1.1

In [15]:
He_states.Z_most_abundant

[1]

In [16]:
He_states.Z_rms

1.224744871391589

We can calculate the properties of the average ionic level.

In [17]:
He_states.average_ion()

CustomParticle(mass=6.645477039375987e-27 kg, charge=1.7623942974e-19 C)

We can use the `summarize()` method to get information about the ionization state.

In [18]:
He_states.summarize()

IonizationState instance for He-4 with Z_mean = 1.10
----------------------------------------------------------------
He-4  0+: 0.100    n_i = 1.00e+12 m**-3
He-4  1+: 0.700    n_i = 7.00e+12 m**-3
He-4  2+: 0.200    n_i = 2.00e+12 m**-3
----------------------------------------------------------------
n_elem = 1.00e+13 m**-3
n_e = 1.10e+13 m**-3
----------------------------------------------------------------


## Ionization states of multiple elements

[Gilbert et al. (2012)]: https://doi.org/10.1088/0004-637X/751/1/20

Now let's look at some actual average ionization state data for filament material in an ICME observed by the *Advanced Composition Explorer* (*ACE*) near 1 AU.  The data were estimated from Figure 4 in [Gilbert et al. (2012)].  These data are noteworthy because there is information from very low charge states to very high charge states.

In [19]:
number_densities = {
    "C": [0, 5.7e-7, 4.3e-5, 3.6e-6, 2.35e-6, 1e-6, 1.29e-6] * u.cm ** -3,
    "O": [0, 1.2e-7, 2.2e-4, 7.8e-6, 8.8e-7, 1e-6, 4e-6, 1.3e-6, 1.2e-7] * u.cm ** -3,
    "Fe": [
        0,
        0,
        1.4e-8,
        1.1e-7,
        2.5e-7,
        2.2e-7,
        1.4e-7,
        1.2e-7,
        2.1e-7,
        2.1e-7,
        1.6e-7,
        8e-8,
        6.3e-8,
        4.2e-8,
        2.5e-8,
        2.3e-8,
        1.5e-8,
        3.1e-8,
        6.1e-9,
        2.3e-9,
        5.3e-10,
        2.3e-10,
        0,
        0,
        0,
        0,
        0,
    ]
    * u.cm ** -3,
}

Now let's use this information as an input for `IonizationStateCollection`: a data structure for the ionization states of multiple elements.

In [20]:
states = IonizationStateCollection(number_densities)

We can index this to get an `IonizationState` for one of the elements.

In [21]:
states["C"]

<IonizationState instance for C>

We can get the relative abundances of each of the elements.

In [22]:
states.abundances

{'C': 0.17942722921968796, 'O': 0.8146086249190311, 'Fe': 0.005964145861281176}

In [23]:
states.log_abundances

{'C': -0.7461116493492604, 'O': -0.08905099599945356, 'Fe': -2.224451743828212}

We can get the number densities as a `dict` (like what we provided) and the electron number density (assuming quasineutrality, but only for the elements contained in the data structure).

In [24]:
states.number_densities

{'C': <Quantity [ 0.  ,  0.57, 43.  ,  3.6 ,  2.35,  1.  ,  1.29] 1 / m3>,
 'O': <Quantity [0.0e+00, 1.2e-01, 2.2e+02, 7.8e+00, 8.8e-01, 1.0e+00, 4.0e+00,
            1.3e+00, 1.2e-01] 1 / m3>,
 'Fe': <Quantity [0.0e+00, 0.0e+00, 1.4e-02, 1.1e-01, 2.5e-01, 2.2e-01, 1.4e-01,
            1.2e-01, 2.1e-01, 2.1e-01, 1.6e-01, 8.0e-02, 6.3e-02, 4.2e-02,
            2.5e-02, 2.3e-02, 1.5e-02, 3.1e-02, 6.1e-03, 2.3e-03, 5.3e-04,
            2.3e-04, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00] 1 / m3>}

In [25]:
states.n_e

<Quantity 638.73093 1 / m3>

We can summarize this information too, but let's specify the minimum ionic fraction to print.

In [27]:
states.summarize(minimum_ionic_fraction=0.1)

IonizationStateCollection instance for: C, O, Fe
----------------------------------------------------------------
C  2+: 0.830    n_i = 4.30e+01 m**-3
----------------------------------------------------------------
O  2+: 0.935    n_i = 2.20e+02 m**-3
----------------------------------------------------------------
Fe  4+: 0.145    n_i = 2.50e-01 m**-3
Fe  5+: 0.128    n_i = 2.20e-01 m**-3
Fe  8+: 0.122    n_i = 2.10e-01 m**-3
Fe  9+: 0.122    n_i = 2.10e-01 m**-3
----------------------------------------------------------------
n_e = 6.39e+02 m**-3
----------------------------------------------------------------


Now let's make some plots!